In [134]:
import transformers as tr

# use own pytorch training. matching saved with noted model weights is probably easier than. also try using own data set

In [135]:
tokenizer = tr.AutoTokenizer.from_pretrained("bert-base-cased")

loading configuration file https://huggingface.co/bert-base-cased/resolve/main/config.json from cache at C:\Users\Ameno/.cache\huggingface\transformers\a803e0468a8fe090683bdc453f4fac622804f49de86d7cecaee92365d4a0f829.a64a22196690e0e82ead56f388a3ef3a50de93335926ccfa20610217db589307
Model config BertConfig {
  "_name_or_path": "bert-base-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.18.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 28996
}

loading file https://huggingface.co/bert-base-cased/resolve

In [136]:
batch_sentences = [
    "But what about second breakfast?",
    "Don't think he knows about second breakfast, Pip.",
    "What about elevensies?",
]
encoded_input = tokenizer(batch_sentences, padding=True, truncation=True, return_tensors="pt")
print(encoded_input)

{'input_ids': tensor([[  101,  1252,  1184,  1164,  1248,  6462,   136,   102,     0,     0,
             0,     0,     0,     0,     0],
        [  101,  1790,   112,   189,  1341,  1119,  3520,  1164,  1248,  6462,
           117, 21902,  1643,   119,   102],
        [  101,  1327,  1164,  5450, 23434,   136,   102,     0,     0,     0,
             0,     0,     0,     0,     0]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]])}


In [137]:
tokenizer.decode(encoded_input["input_ids"][0])

'[CLS] But what about second breakfast? [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]'

In [138]:
from datasets import load_dataset

dataset = load_dataset("yelp_review_full")
dataset["train"][100]

Reusing dataset yelp_review_full (C:\Users\Ameno\.cache\huggingface\datasets\yelp_review_full\yelp_review_full\1.0.0\e8e18e19d7be9e75642fc66b198abadb116f73599ec89a69ba5dd8d1e57ba0bf)














100%|██████████| 2/2 [00:00<00:00,  9.10it/s]


{'label': 0,
 'text': 'My expectations for McDonalds are t rarely high. But for one to still fail so spectacularly...that takes something special!\\nThe cashier took my friends\'s order, then promptly ignored me. I had to force myself in front of a cashier who opened his register to wait on the person BEHIND me. I waited over five minutes for a gigantic order that included precisely one kid\'s meal. After watching two people who ordered after me be handed their food, I asked where mine was. The manager started yelling at the cashiers for \\"serving off their orders\\" when they didn\'t have their food. But neither cashier was anywhere near those controls, and the manager was the one serving food to customers and clearing the boards.\\nThe manager was rude when giving me my order. She didn\'t make sure that I had everything ON MY RECEIPT, and never even had the decency to apologize that I felt I was getting poor service.\\nI\'ve eaten at various McDonalds restaurants for over 30 years. 

In [139]:
tokenized_datasets = dataset.map(lambda x: tokenizer(x['text'], padding="max_length", truncation=True), batched=True)

Loading cached processed dataset at C:\Users\Ameno\.cache\huggingface\datasets\yelp_review_full\yelp_review_full\1.0.0\e8e18e19d7be9e75642fc66b198abadb116f73599ec89a69ba5dd8d1e57ba0bf\cache-1212de8c9c92504b.arrow
Loading cached processed dataset at C:\Users\Ameno\.cache\huggingface\datasets\yelp_review_full\yelp_review_full\1.0.0\e8e18e19d7be9e75642fc66b198abadb116f73599ec89a69ba5dd8d1e57ba0bf\cache-7a0846779351e1d6.arrow


In [140]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(16))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(16))

Loading cached shuffled indices for dataset at C:\Users\Ameno\.cache\huggingface\datasets\yelp_review_full\yelp_review_full\1.0.0\e8e18e19d7be9e75642fc66b198abadb116f73599ec89a69ba5dd8d1e57ba0bf\cache-385f4f630def7870.arrow
Loading cached shuffled indices for dataset at C:\Users\Ameno\.cache\huggingface\datasets\yelp_review_full\yelp_review_full\1.0.0\e8e18e19d7be9e75642fc66b198abadb116f73599ec89a69ba5dd8d1e57ba0bf\cache-5550b17c557af15e.arrow


In [141]:
small_train_dataset

Dataset({
    features: ['label', 'text', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 16
})

In [142]:
from transformers import AutoModelForSequenceClassification

# Fetches the pre- trained model of BERT, removes the head and replaces it with a classification head with specified output nodes 
model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=5)

# Freeze base model parameters
for param in model.base_model.parameters():
    param.requires_grad = False

loading configuration file https://huggingface.co/bert-base-cased/resolve/main/config.json from cache at C:\Users\Ameno/.cache\huggingface\transformers\a803e0468a8fe090683bdc453f4fac622804f49de86d7cecaee92365d4a0f829.a64a22196690e0e82ead56f388a3ef3a50de93335926ccfa20610217db589307
Model config BertConfig {
  "_name_or_path": "bert-base-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 1

In [143]:
from transformers import TrainingArguments, Trainer


# DO THIS MANUALLY WITH PYTORCH AFTERWARDS 
training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch")

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [144]:
import numpy as np
from datasets import load_metric

metric = load_metric("accuracy")

In [145]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [146]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset, 
    compute_metrics=compute_metrics,
)

In [147]:
trainer.train()

C:\Users\Ameno\Anaconda3\envs\luna-ai\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 16
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 6





















***** Running Evaluation *****
  Num examples = 16
  Batch size = 8


















                               


                            




                      





                                     
                                  



                         






                







                     








                  











 50%|█████     | 1/2 [2:53:04<2:53:04

{'eval_loss': 1.7006750106811523, 'eval_accuracy': 0.125, 'eval_runtime': 22.573, 'eval_samples_per_second': 0.709, 'eval_steps_per_second': 0.089, 'epoch': 1.0}
















***** Running Evaluation *****
  Num examples = 16
  Batch size = 8


















                                         


                                      




                               





                            








                                                    
                                            



                                  






                         







                     



















 50%|█████     | 1/2 [2:53:53<2:53:53, 10433.00s/it]

























{'eval_loss': 1.6964863538742065, 'eval_accuracy': 0.125, 'eval_runtime': 23.359, 'eval_samples_per_second': 0.685, 'eval_steps_per_second': 0.086, 'epoch': 2.0}
















***** Running Evaluation *****
  Num examples = 16
  Batch size = 8


















                                         


                                      




                               





                                                    
                                            



                                   






                         







                     








                  











 50%|█████     | 1/2 [2:54:51<2:54:51, 10491.92s/it]

































Training completed. Do not forget to share your model on huggingface.co/models =)




                                         


                                      




                               





                                                    
                                            



                                   






                         







                     











 50%|█████     | 1/2 [2:54:52<2:54:

{'eval_loss': 1.6947609186172485, 'eval_accuracy': 0.125, 'eval_runtime': 28.883, 'eval_samples_per_second': 0.554, 'eval_steps_per_second': 0.069, 'epoch': 3.0}
{'train_runtime': 158.704, 'train_samples_per_second': 0.302, 'train_steps_per_second': 0.038, 'train_loss': 1.7364095052083333, 'epoch': 3.0}


TrainOutput(global_step=6, training_loss=1.7364095052083333, metrics={'train_runtime': 158.704, 'train_samples_per_second': 0.302, 'train_steps_per_second': 0.038, 'train_loss': 1.7364095052083333, 'epoch': 3.0})

In [148]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 16
  Batch size = 8














100%|██████████| 2/2 [00:14<00:00,  7.39s/it]


{'eval_loss': 1.6947609186172485,
 'eval_accuracy': 0.125,
 'eval_runtime': 29.945,
 'eval_samples_per_second': 0.534,
 'eval_steps_per_second': 0.067,
 'epoch': 3.0}

In [149]:
import torch as ts

ts.save(model.state_dict(), "../model/bert.pt")

In [150]:
class hugBert(ts.nn.Module):
  def __init__(self, num_classes):
    super(hugBert, self).__init__()
    device = ts.device('cuda' if ts.cuda.is_available() else 'cpu')
    self.bert = tr.BertModel.from_pretrained('bert-base-cased', return_dict=True).to(device)
    self.classifier = ts.nn.Linear(768, num_classes)#, bias=False)
  
  # def forward(self, *args, **kwargs):
    
    
  def forward(self, input_ids, token_type_ids, attention_mask, labels):
    # outputs = self.bert(*args, **kwargs)
    print("im here")
    outputs = self.bert(input_ids, token_type_ids=token_type_ids, attention_mask=attention_mask)

    # print("im herenow", outputs)
    pred = self.classifier(outputs[1])#outputs.pooler_output)
    print("pred:",pred)
    return pred

device = ts.device('cuda' if ts.cuda.is_available() else 'cpu')
myModel = hugBert(5).to(device)


loading configuration file https://huggingface.co/bert-base-cased/resolve/main/config.json from cache at C:\Users\Ameno/.cache\huggingface\transformers\a803e0468a8fe090683bdc453f4fac622804f49de86d7cecaee92365d4a0f829.a64a22196690e0e82ead56f388a3ef3a50de93335926ccfa20610217db589307
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.18.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 28996
}

loading weights file https://huggingface.co/bert-base-cased/resolve/main/pytorch_model.bin from c

In [151]:
myModel.load_state_dict(ts.load("../model/bert.pt"))

<All keys matched successfully>

In [152]:
myModel.eval()
training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch")
evaluator = Trainer(
    model=myModel,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset, 
    compute_metrics=compute_metrics,
)
evaluator.evaluate()

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
***** Running Evaluation *****
  Num examples = 16
  Batch size = 8


im here
pred: tensor([[ 0.1826,  0.6449,  0.0217, -0.0519,  0.0404],
        [ 0.1297,  0.5231,  0.1399,  0.0172,  0.0406],
        [ 0.1245,  0.3989,  0.2945, -0.1136,  0.1910],
        [ 0.1148,  0.4368,  0.1891, -0.1092,  0.2012],
        [ 0.0477,  0.4002,  0.1700, -0.0738,  0.2075],
        [ 0.1610,  0.5558,  0.1638, -0.0263,  0.1625],
        [ 0.1027,  0.6949,  0.0231,  0.0599, -0.0183],
        [-0.0012,  0.4984,  0.1035, -0.0801,  0.1947]])


im here


pred: tensor([[ 0.1858,  0.5179,  0.1967, -0.0876,  0.1718],
        [ 0.1486,  0.4860,  0.1535, -0.0523,  0.1371],
        [ 0.1686,  0.5517,  0.1913, -0.0541,  0.1021],
        [ 0.0683,  0.4326,  0.2662, -0.1737,  0.2609],
        [ 0.1184,  0.4148,  0.2677, -0.1726,  0.2505],
        [ 0.1091,  0.4781,  0.2054, -0.1284,  0.2082],
        [ 0.1833,  0.4260,  0.1584, -0.1261,  0.1859],
        [ 0.0658,  0.4718,  0.2040, -0.0485,  0.2178]])


ValueError: Mismatch in the number of predictions (14) and references (16)